In [154]:
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline
from scipy import stats

Columns (1,14) have mixed types.

In [155]:
fires = pd.read_csv('Historical_DOB_Permit_Issuance(1).csv', low_memory = False)

In [156]:
fires.shape

(2428526, 60)

What fraction of all construction permits in this data set correspond to renewed permits? The filing type is recorded under column 'Filing Status'. For first-time permits the value in this column is 'INITIAL' and for renewed permits it is 'RENEWAL'.

In [157]:
#Exploring the columns
print(list(fires.columns))

['BOROUGH', 'BIN', 'Number', 'Street', 'Job #', 'Job doc. #', 'Job Type', 'Self_Cert', 'Block', 'Lot', 'Community Board', 'Postcode', 'Bldg Type', 'Residential', 'Special District 1', 'Special District 2', 'Work Type', 'Permit Status', 'Filing Status', 'Permit Type', 'Permit Sequence #', 'Permit Subtype', 'Oil Gas', 'Site Fill', 'Filing Date', 'Issuance Date', 'Expiration Date', 'Job Start Date', "Permittee's First Name", "Permittee's Last Name", "Permittee's Business Name", "Permittee's Phone #", "Permittee's License Type", "Permittee's License #", 'Act as Superintendent', "Permittee's Other Title", 'HIC License', "Site Safety Mgr's First Name", "Site Safety Mgr's Last Name", 'Site Safety Mgr Business Name', 'Superintendent First & Last Name', 'Superintendent Business Name', "Owner's Business Type", 'Non-Profit', "Owner's Business Name", "Owner's First Name", "Owner's Last Name", "Owner's House #", "Owner's House Street Name", 'Owner’s House City', 'Owner’s House State', 'Owner’s Hous

In [158]:
fires['Filing Status'].value_counts()

INITIAL    1851583
RENEWAL     576943
Name: Filing Status, dtype: int64

In [159]:
total=sum(fires['Filing Status'].value_counts())

In [160]:
renewal=fires['Filing Status'].value_counts()[1]


In [161]:
renewal/total

0.23756920864755

In [162]:
renewals=pd.DataFrame(fires[fires['Filing Status']=='RENEWAL']).reset_index(drop=True)

In [163]:
renewals["Owner's Business Type"].value_counts()

CORPORATION           186449
INDIVIDUAL            172545
PARTNERSHIP           110741
OTHER                  51612
OTHER GOV'T AGENCY      9614
CONDO/CO-OP             6405
NYCHA                   2596
HPD                      848
DCAS                     333
NY STATE                 287
HHC                      279
DOE                      220
NYC AGENCY                28
NYCHA/HHC                  5
Name: Owner's Business Type, dtype: int64

186449:172545

In [164]:
186449/172545

1.0805818771914573

Consider only those permits that were issued for more than 365 days. The date on which a permit was issued is recorded in column 'Issuance Date', the date the permit expires is in column 'Expiration Date'. What fraction of these at least year-long permits were issued in the borough with the highest number of such permits? The borough where the construction will take place is in column 'BOROUGH'.

In [165]:
fires['Issuance Date']

0          2010-11-05T00:00:00
1          2012-01-30T00:00:00
2          2008-02-04T00:00:00
3          1998-08-31T00:00:00
4          2007-04-30T00:00:00
                  ...         
2428521    2003-10-08T00:00:00
2428522    1997-07-28T00:00:00
2428523    1999-07-09T00:00:00
2428524    1996-06-25T00:00:00
2428525    1999-09-20T00:00:00
Name: Issuance Date, Length: 2428526, dtype: object

In [166]:
fires.dropna(subset=['Expiration Date', 'Issuance Date'],inplace=True)

In [167]:
len(fires)

2428522

In [168]:
expiration_digital = []
for t in fires['Expiration Date']:
    try:
        expiration_digital.append(datetime.strptime(t,'%Y-%m-%dT00:00:00'))
        
    except ValueError:
        print('val')
        expiration_digital.append(np.nan)
    except TypeError:
        print('type')
        expiration_digital.append(np.nan)


In [169]:
issuance_digital = []
for t in fires['Issuance Date']:
    try:
        issuance_digital.append(datetime.strptime(t,'%Y-%m-%dT00:00:00'))
        
    except ValueError:
        print('val')
        issuance_digital.append(np.nan)
    except TypeError:
        print('type')
        issuance_digital.append(np.nan)
  

In [170]:
fires['expiration_digital']=expiration_digital

In [171]:
fires['issuance_digital']=issuance_digital

In [173]:
check=fires[['Expiration Date','expiration_digital']]

In [175]:
check.head()

,Expiration Date,expiration_digital
0,2011-11-05T00:00:00,2011-11-05
1,2013-01-29T00:00:00,2013-01-29
2,2009-02-03T00:00:00,2009-02-03
3,1999-08-31T00:00:00,1999-08-31
4,2008-01-08T00:00:00,2008-01-08


In [176]:
fires['permit_duration']=fires['expiration_digital']-fires['issuance_digital']

In [177]:
fires['test']=fires['permit_duration'].apply(lambda x: x.days>365)

In [178]:
fires['test']

0          False
1          False
2          False
3          False
4          False
           ...  
2428521    False
2428522    False
2428523    False
2428524    False
2428525    False
Name: test, Length: 2428522, dtype: bool

In [179]:
long=pd.DataFrame(fires[fires['test']==1]).reset_index(drop=True)

In [180]:
long.head(40)

,BOROUGH,BIN,Number,Street,Job #,Job doc. #,Job Type,Self_Cert,Block,Lot,...,Latitude,Longitude,Council District,Census Tract,BBL,NTA,expiration_digital,issuance_digital,permit_duration,test
0,BRONX,2114490,1,EAST 161 STREET,210043825,3,A2,NaN,2493,1,...,40.829035,-73.928497,16.0,63.0,2.024930e+09,West Concourse ...,2010-09-06,2008-04-25,864 days,True
1,BRONX,2014132,2614,JEROME AVENUE,200086922,1,A1,NaN,3191,20,...,40.866834,-73.897884,14.0,401.0,2.031910e+09,Bedford Park-Fordham North ...,1992-03-13,1991-03-13,366 days,True
2,BRONX,2090312,3469,EAST TREMONT AVENUE,200111681,1,SG,NaN,5419,55,...,40.831884,-73.827266,13.0,164.0,2.054190e+09,Schuylerville-Throgs Neck-Edgewater Park ...,1992-05-23,1991-05-23,366 days,True
3,BRONX,2095398,120,BELLAMY LOOP,200717947,2,A2,NaN,5141,120,...,40.872875,-73.826433,12.0,46201.0,2.051410e+09,Co-op City ...,2004-12-31,2003-12-29,368 days,True
4,BRONX,2114490,1,EAST 161 STREET,210041701,2,A2,NaN,2493,1,...,40.829035,-73.928497,16.0,63.0,2.024930e+09,West Concourse ...,2010-09-06,2008-04-17,872 days,True
5,BRONX,2009815,1394,CROTONA AVENUE,200232195,2,A1,NaN,2937,7,...,40.833395,-73.898058,16.0,151.0,2.029370e+09,Morrisania-Melrose ...,1995-11-19,1993-11-19,730 days,True
6,BRONX,2096807,100,CASALS PLACE,200717956,2,A2,NaN,5141,120,...,40.878457,-73.824162,12.0,46201.0,2.051410e+09,Co-op City ...,2004-12-31,2003-12-29,368 days,True
7,BRONX,2008180,1723,TOWNSEND AVENUE,200259371,2,A1,NaN,2849,28,...,40.846738,-73.911899,14.0,22701.0,2.028490e+09,Mount Hope ...,1995-11-22,1993-11-22,730 days,True
8,BRONX,2101839,308,OLMSTEAD AVENUE,200603622,3,NB,NaN,3483,85,...,40.814234,-73.850043,18.0,4.0,2.034830e+09,Soundview-Castle Hill-Clason Point-Harding Par...,2003-11-19,2002-11-13,371 days,True
9,BRONX,2059979,3433,BOSTON ROAD,200483529,2,A2,NaN,4713,40,...,40.876052,-73.849404,12.0,386.0,2.047130e+09,Eastchester-Edenwald-Baychester ...,2000-01-28,1998-08-14,532 days,True


In [181]:
long['BOROUGH'].value_counts()

MANHATTAN        70177
BROOKLYN         33431
QUEENS           28535
BRONX            12126
STATEN ISLAND    11893
Name: BOROUGH, dtype: int64

In [182]:
70177/sum(long['BOROUGH'].value_counts())

0.44938589413557717

Limit your analysis to permits that were filed in 2010. The column recording the date of permit filing is called 'Filing Date'. For each ZIP code compute the ratio between the number of permits issued in 2010 and number of residents in that ZIP code. Make sure to discard any ZIP codes with population lower than 1000 people. Note: you will need to use both the DOB permits dataset and the US Census dataset. In the DOB permits dataset, the ZIP code is recorded under column 'Postcode'. How many ZIP codes are outliers in terms of number of construction permits per number of people - more specificaly, for how many ZIP codes does this computed ratio exceed the mean by more than twice the standard deviation?

In [183]:
fires['Filing Date']

0          2010-11-05T00:00:00
1          2012-01-30T00:00:00
2          2008-02-04T00:00:00
3          1998-08-31T00:00:00
4          2007-04-30T00:00:00
                  ...         
2428521    2003-10-08T00:00:00
2428522    1996-07-29T00:00:00
2428523    1999-07-09T00:00:00
2428524    1996-06-25T00:00:00
2428525    1999-09-20T00:00:00
Name: Filing Date, Length: 2428522, dtype: object

In [184]:
fires['Filing Year']=fires['Filing Date'].apply(lambda x:str(x).split('-')[0])

In [185]:
y2010=pd.DataFrame(fires[fires['Filing Year']=='2010']).reset_index(drop=True)

In [186]:
pops = pd.read_csv('2010+Census+Population+By+Zipcode+(ZCTA).csv',dtype=str)

In [187]:
pops.head()

,Zip Code ZCTA,2010 Census Population
0,01001,16769
1,01002,29049
2,01003,10372
3,01005,5079
4,01007,14649


In [188]:
y2010grp=y2010.groupby(['Postcode'])['BOROUGH'].count().reset_index()

In [189]:
y2010grp['Postcode']=y2010grp['Postcode'].apply(lambda x: str(int(x)))

In [191]:
y2010grp.head()

,Postcode,PermitCount
0,0,1
1,10000,14
2,10001,1956
3,10002,1564
4,10003,2949


In [192]:
y2010grp.columns=['Postcode','PermitCount']

In [193]:
y2010merge = y2010grp.astype(str).merge(pops,left_on='Postcode',right_on='Zip Code ZCTA',how='left')

In [194]:
y2010merge.dropna(inplace=True)

In [195]:
y2010merge=pd.DataFrame(y2010merge[y2010merge['2010 Census Population'].astype(int)>=1000]).reset_index(drop=True)

In [196]:
y2010merge['ratio']=(y2010merge['PermitCount'].astype(int))/(y2010merge['2010 Census Population'].astype(int))

In [197]:
y2010merge.head()

,Postcode,PermitCount,Zip Code ZCTA,2010 Census Population,ratio
0,10001,1956,10001,21102,0.092693
1,10002,1564,10002,81410,0.019211
2,10003,2949,10003,56024,0.052638
3,10004,464,10004,3089,0.150210
4,10005,467,10005,7135,0.065452


In [198]:
means=np.mean(y2010merge['ratio'])

In [199]:
std=np.std(y2010merge['ratio'])

In [200]:
y2010merge['mean']=means

In [201]:
y2010merge['std']=std

In [202]:
y2010merge.head()

,Postcode,PermitCount,Zip Code ZCTA,2010 Census Population,ratio,mean,std
0,10001,1956,10001,21102,0.092693,0.022365,0.032695
1,10002,1564,10002,81410,0.019211,0.022365,0.032695
2,10003,2949,10003,56024,0.052638,0.022365,0.032695
3,10004,464,10004,3089,0.150210,0.022365,0.032695
4,10005,467,10005,7135,0.065452,0.022365,0.032695


In [203]:
y2010merge['outlier']=(abs(y2010merge['ratio']-y2010merge['mean'])/y2010merge['std'])>2

In [204]:
y2010merge.head(40)

,Postcode,PermitCount,Zip Code ZCTA,2010 Census Population,ratio,mean,std,outlier
0,10001,1956,10001,21102,0.092693,0.022365,0.032695,True
1,10002,1564,10002,81410,0.019211,0.022365,0.032695,False
2,10003,2949,10003,56024,0.052638,0.022365,0.032695,False
3,10004,464,10004,3089,0.150210,0.022365,0.032695,True
4,10005,467,10005,7135,0.065452,0.022365,0.032695,False
5,10006,295,10006,3011,0.097974,0.022365,0.032695,True
6,10007,779,10007,6988,0.111477,0.022365,0.032695,True
7,10009,1307,10009,61347,0.021305,0.022365,0.032695,False
8,10010,1527,10010,31834,0.047968,0.022365,0.032695,False
9,10011,2826,10011,50984,0.055429,0.022365,0.032695,False


In [205]:
sum(y2010merge['outlier'])

9

In [206]:
####

In [207]:
fires['Issuance Year']=fires['Issuance Date'].apply(lambda x:str(x).split('-')[0])

In [208]:
limited=fires[(fires['Issuance Year'].astype(int) >= 1990)&(fires['Issuance Year'].astype(int) <= 2012)]

In [209]:
yearcounts=pd.DataFrame(limited['Issuance Year'].value_counts()).reset_index()

In [210]:
yearcounts.columns=['year','counts']

In [211]:
X = yearcounts['year'].values.reshape(-1,1)
y = yearcounts['counts'].values.reshape(-1,1)

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [212]:
from sklearn import linear_model

regr = linear_model.LinearRegression()  
regr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [214]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X,y))


Coefficients: 
 [[5851.73616601]]

Intercept: 
 [-11605968.41600791]

R-squared:
0.9068172786023697


In [215]:
jobs=pd.DataFrame(fires['Job Start Date'])
jobs['permit_duration']=fires['permit_duration']

In [216]:
jobs.dropna(inplace=True)

In [217]:
month=[]
for c,i in enumerate(jobs['Job Start Date']):
    try:
        start=i.split('-')[1]
        month.append(start)
    except:
        month.append(np.nan)

In [218]:
jobs['start_month']=month

In [219]:
jobs.dropna(inplace=True)

In [220]:
jobs['start_year']=jobs['Job Start Date'].apply(lambda x: x.split('-')[0])

In [221]:
jobs=jobs[(jobs['start_year'].astype(int) >= 1990)&(jobs['start_year'].astype(int) <= 2012)]

In [222]:
jobs['short']=jobs['permit_duration'].apply(lambda x: np.where(x.days<60,1,-1)) 

In [223]:
jobs['nov']=jobs['start_month'].apply(lambda x: np.where(x=='11',1,-1))

In [224]:
jobs.head()

,Job Start Date,permit_duration,start_month,start_year,short,nov
0,2010-11-05T00:00:00,365 days,11,2010,-1,1
1,2002-08-08T00:00:00,365 days,08,2002,-1,-1
2,2005-08-29T00:00:00,365 days,08,2005,-1,-1
3,1998-08-31T00:00:00,365 days,08,1998,-1,-1
4,2007-04-30T00:00:00,253 days,04,2007,-1,-1


In [225]:
notshort=pd.DataFrame(jobs[jobs['short']==-1]).reset_index(drop=True)

In [226]:
notshorty=notshort.groupby(['nov'])['short'].count().reset_index()
notshorty.columns=['nov','notshort']

In [227]:
short=pd.DataFrame(jobs[jobs['short']==1]).reset_index(drop=True)

In [228]:
shorty=short.groupby(['nov'])['short'].count().reset_index()

In [229]:
notshorty.head()

,nov,notshort
0,-1,2047704
1,1,172706


In [230]:
shorty.head()

,nov,short
0,-1,150609
1,1,21103


In [231]:
shorty['notshort']=notshorty['notshort']

In [232]:
shorty.head()

,nov,short,notshort
0,-1,150609,2047704
1,1,21103,172706


In [233]:
table=[list(shorty.iloc[0])[1:],list(shorty.iloc[1])[1:]]

In [234]:
table

[[150609, 2047704], [21103, 172706]]

In [235]:
import scipy
#table=[[150609, 2047704], [21103, 172706]]
stat, p, dof, expected = stats.chi2_contingency(table)


In [236]:
stat

4356.766804329866

In [237]:
p

0.0

In [238]:
dof

1

In [239]:
expected

array([[ 157799.94576196, 2040513.05423804],
       [  13912.05423804,  179896.94576196]])

Let's investigate how the number of construction jobs that start in the summer vs the winter changes across the years. The date of construction job start is recorded in column 'Job Start Date'. For every year, compute the ratio between the number of construction jobs that start in the peak of summer (in July and August) and the number of jobs that start in the peak of winter (in January and February). Again limit your analysis to years 1990-2012 (both inclusive). Find the year when this ratio was maximal - what was the value of this ratio for that year?

In [240]:
jobs['summer']=np.where(jobs['start_month'].isin(['07','08']),1,0)

In [241]:
jobs['winter']=np.where(jobs['start_month'].isin(['01','02']),1,0)

In [242]:
jobs['sumwint']=jobs['summer']+jobs['winter']

In [243]:
jobseason=jobs.groupby(['start_year'])['summer','winter'].sum().reset_index()

<ipython-input-243-9c29deb57d0a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  jobseason=jobs.groupby(['start_year'])['summer','winter'].sum().reset_index()


In [244]:
jobseason.head()

,start_year,summer,winter
0,1990,5989,1574
1,1991,8719,6486
2,1992,9647,8468
3,1993,11044,8417
4,1994,10922,8378


In [245]:
jobseason['ratio']=jobseason['summer']/jobseason['winter']

In [246]:
jobseason.head(40)

,start_year,summer,winter,ratio
0,1990,5989,1574,3.804956
1,1991,8719,6486,1.344280
2,1992,9647,8468,1.139230
3,1993,11044,8417,1.312106
4,1994,10922,8378,1.303652
5,1995,10882,9553,1.139119
6,1996,12810,9026,1.419233
7,1997,14106,11056,1.275868
8,1998,15559,12691,1.225987
9,1999,17639,13502,1.306399


In [247]:
max(jobseason['ratio'])

3.8049555273189326

In [ ]:
#Find columns with less than half nulls and keep these.
#null_count = fires.isnull().sum()
#null_count = pd.DataFrame(null_count[null_count<1138890])
#nullist = list(null_count.index)
#nullist

In [ ]:
#fires2 = fires[nullist]
#fires2.head()

In [ ]:
#look at high-null columns still included in database
#null_count2 = fires2.isnull().sum()
#null_count2 = pd.DataFrame(null_count2[null_count2>45000])
#list(null_count2.index)

We are going to play a simple game. We start with 0 points. We will flip a coin. If it comes up heads, we get a point. If comes up tails, we double our current score.

For example, if we got HTH, we would have ((0+1)∗2+1)=3 points, but if we got THT, we would have ((0)∗2+1)∗2=2

points.
Let's start with a fair coin. What is our expected score after 5 flips?

In [248]:
import itertools
import numpy as np


In [249]:
import json
results=list((itertools.product("HT", repeat=5)))

In [250]:
def cointoss(perms):
    
    for i in perms:
        total=0
        for j in i:

            if j=='H':
                total+=1
            if j=='T':
                total=total*2

        yield total

In [276]:
for num in range(1,5):
    #num=3
    results=list((itertools.product("H"+"T"*num, repeat=10)))
    expected=sum(cointoss(results))/len(results)
    print(expected,"H"+"T"*num)

56.6650390625 HT
82.19085843960102 HTT
89.46312999725342 HTTT
89.0116806656 HTTTT


In [275]:
np.std(list(cointoss(results)))

29.252068059034414

In [268]:
"HTTT"
89.46312999725342

89.46312999725342

In [269]:
power2=[2**i for i in list(range(1,9))]

In [271]:
prob=0
count=0
for i in list(cointoss(results)):
    count+=1
    if i in power2:
        prob+=1

In [272]:
prob/count

0.072265625

In [ ]:
#Find non-numeric columns
#cat = fires.select_dtypes(include=['object'])
#for i in cat:
   #column = cat[i]
   #print(i)
   #print(column.nunique())

In [ ]:
#fires3 = fires2.dropna()

In [ ]:
fires4 = fires
fires4['FIRE_BOX'] = fires4['FIRE_BOX'].replace('NYCHA','')
fires4.head()

In [ ]:
fires4['short_zip'] = (fires4['ZIP_CODE'].replace('-',''))

In [ ]:
fires4['short_zip'] = fires4['short_zip'].apply(lambda x: x[0:5] if type(x)==str else np.nan)

In [ ]:
arrival_digital = []
for t in fires4['ARRIVAL_DATE_TIME']:
    try:
        arrival_digital.append(datetime.strptime(t,'%m/%d/%Y %I:%M:%S %p'))
        
    except ValueError:
        arrival_digital.append(np.nan)
    except TypeError:
        arrival_digital.append(np.nan)

arrival_digital   

In [ ]:
incident_digital = []
for t in fires4['INCIDENT_DATE_TIME']:
    try:
        incident_digital.append(datetime.strptime(t,'%m/%d/%Y %I:%M:%S %p'))
        
    except ValueError:
        incident_digital.append(np.nan)
    except TypeError:
        incident_digital.append(np.nan)

arrival_digital   

In [ ]:
fires4['arrival_digital'] = arrival_digital
fires4['incident_digital']= incident_digital

In [ ]:
(fires4['INCIDENT_TYPE_DESC'].value_counts()[0])/2277779

In [ ]:
falsecalls = fires4[fires4['INCIDENT_TYPE_DESC'] == '710 - Malicious, mischievous false call, other']

In [ ]:
falsecallsgroup = falsecalls.groupby(['BOROUGH_DESC'])['BOROUGH_DESC'].count()

In [ ]:
falsecallsgroup

In [ ]:
8276/25240

In [ ]:
incedents = fires4.groupby(['INCIDENT_TYPE_DESC'])['UNITS_ONSCENE'].mean()

In [ ]:
incedents[100:150]

In [ ]:
11.084640/ 4.016524

In [ ]:
buildingfire = pd.DataFrame(fires4[fires4['INCIDENT_TYPE_DESC'] ==  '111 - Building fire'])

In [ ]:
buildingfire['time_lapse_sec']= (buildingfire['arrival_digital'] - buildingfire['incident_digital'])

In [ ]:
buildingfire['time_lapse_sec']= buildingfire['time_lapse_sec'].apply(lambda x: x.total_seconds())

In [ ]:
buildingfire.head()

In [ ]:
buildingfire['time_lapse_sec'].quantile([0.25,0.5,0.75])

In [ ]:
249/60

In [ ]:
pops = pd.read_csv('2010+Census+Population+By+Zipcode+(ZCTA).csv')

In [ ]:
pops.head()

In [ ]:
bfire = pd.DataFrame(buildingfire.groupby(['FIRE_BOX'])['FIRE_BOX'].count())

In [ ]:
bfire.columns = ['tot_built_fires']
bfire = bfire.reset_index()
bfire['FIRE_BOX']=bfire['FIRE_BOX'].astype(int)
bfire = pd.DataFrame(bfire.dropna())

In [ ]:
merge_bfire = bfire.merge(pops,left_on='FIRE_BOX',right_on='Zip Code ZCTA')

In [ ]:
merge_bfire.head()

In [ ]:
((merge_bfire['tot_built_fires'].corr(merge_bfire['2010 Census Population']))**2)*100

In [ ]:
cooking = pd.DataFrame(fires4[fires4['INCIDENT_TYPE_DESC']=='113 - Cooking fire, confined to container'])

In [ ]:
cooking.head()

In [ ]:
cooking['incident_hour'] = cooking['INCIDENT_DATE_TIME'].apply(lambda x: x.split()[1] if x==x else np.nan)

In [ ]:
cooking['am_pm'] = cooking['INCIDENT_DATE_TIME'].apply(lambda x: x.split()[2] if x==x else np.nan)

In [ ]:
cooking['incident_hour'] = cooking['incident_hour'].apply(lambda x: x.split(':')[0] if x==x else np.nan)

In [ ]:
cooking['incident_hour'] = cooking['incident_hour'] + ' ' + cooking['am_pm']

In [ ]:
cooking.head()

In [ ]:
cooking_group = pd.DataFrame(cooking.groupby(['incident_hour']).count()).reset_index()
cooking_group.head()

In [ ]:
cooking_group.shape

In [ ]:
cooking_group.sort_values(by = 'FIRE_BOX',axis = 0,ascending = False)

In [ ]:
6806/np.sum(cooking_group['FIRE_BOX'])

In [ ]:
CO = pd.DataFrame(fires4.dropna(subset=['CO_DETECTOR_PRESENT_DESC']))

In [ ]:
CO['duration_min'] = CO['TOTAL_INCIDENT_DURATION'].apply(lambda x: x/60)
CO.shape

In [ ]:
bins = pd.IntervalIndex.from_tuples([(20, 30), (30, 40), (40, 50),(50,60),(60,70)])
dur_intmin = pd.DataFrame(pd.cut(CO['duration_min'], bins))

In [ ]:
dur_intmin.head()

In [ ]:
CO['dur_intmin']=dur_intmin['duration_min']

In [ ]:
CO.head()

In [ ]:
COdetector = pd.DataFrame(None)
COdetector['present']= CO['CO_DETECTOR_PRESENT_DESC']

In [ ]:
COdetector['int_min'] = CO['dur_intmin']
COdetector.shape

In [ ]:
COdetector['int'] = COdetector['int_min'].apply(lambda x: str(x))

In [ ]:
int_count = (pd.DataFrame(COdetector.groupby(['int']).count()).reset_index()).drop(columns = ['present'])
int_count.columns = ['int','int_num']
int_count.head()

In [ ]:
COdetect = pd.DataFrame(COdetector.groupby(['int','present']).count()).reset_index()
COdetect.columns = ['int','present','num']
COdetect.head()

In [ ]:
CO_count = COdetect.merge(int_count,on = 'int',how = 'left')
CO_count.head()

In [ ]:
CO_count['proportion'] = CO_count['num']/CO_count['int_num']
CO_count

In [ ]:
CO_count['target_min'] = CO_count['int'].apply(lambda x: int(x.split(',')[0][1:])+5)

In [ ]:
CO_count

In [ ]:
CO_count['detector'] = np.where(CO_count['present']=='Yes',1,0)
CO_count.head()

In [ ]:
ratio = []
for i in range(0,len(CO_count['proportion'])):
    if CO_count['detector'][i]==0:
        ratio.append(float(CO_count['proportion'][i]/CO_count['proportion'][i+1]))
ratio   

In [ ]:
minutes = pd.DataFrame(CO_count.groupby('target_min').count()).reset_index()
minutes['ratio'] = ratio
minutes2 = minutes.dropna()
minutes2

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
Y = CO_count['proportion']
X = CO_count.drop(columns = ['int','num','int_num','present','proportion'])
regr.fit(X, Y)

In [ ]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

In [ ]:
no = 39*1.26143519e-19 + 0.27707225591998497 +0

In [ ]:
yes = 39*1.26143519e-19 + 0.27707225591998497 + 4.45855488e-01

In [ ]:
result = no/yes
result

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
Y = minutes['ratio'].ravel()
X = (minutes['target_min'].ravel()).reshape(-1,1)
regr.fit(X, Y)

In [ ]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

In [ ]:
predict = regr.predict(X)

In [ ]:
print('Predicted ratio:{}'.format(0.01225494*39 -0.14705871327881903))

In [ ]:
CO['long_time'] = np.where(CO['duration_min']>60,1,-1) 


In [ ]:
CO['present'] = np.where(CO['CO_DETECTOR_PRESENT_DESC']=='NO',1,-1)
CO.head()

In [ ]:
stats.chisquare(CO['present'], CO['long_time'])